In [ ]:
# PyTorch imports
import torch  # Main PyTorch library
import torch.nn as nn  # Neural network modules
import torch.optim as optim  # Optimization algorithms
#import torch.nn.functional as F  # Activation functions and other functional operations

import numpy as np # For matrix operations

# Data handling imports
from torch.utils.data import Dataset, DataLoader, random_split  # For dataset operations and data loading
from torchvision import datasets, transforms  # For vision datasets and image transformations

# Visualization and utility imports
import matplotlib.pyplot as plt  # For plotting graphs and images
import numpy as np  # For numerical operations
from tqdm import tqdm  # For progress bars in loops